In [187]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [188]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")

In [189]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.65, test_size=0.35, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.571, random_state=42)

# Above two lines in unison accomplish a 65-15-20 split for train-test-validation

In [194]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 64
val_batch_size = 64
test_batch_size = 64

cpu
cpu


In [195]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        df.iloc[:, [-2, -1]] = df.iloc[:, [-1, -2]].values
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1], dtype=torch.long)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
print("X_train", X_train)
print("y_train", y_train)
print("train_df:", train_df)
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [196]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc5 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc6 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc6(x)
        
        return x


In [197]:
import optuna

num_epochs = 3
trial_count = 1

learning_rate = 0.01
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += 1
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.unsqueeze(1))
                val_loss += loss.float().item()
                count += 1
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 10, 100)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.9)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

study = optuna.create_study(direction="minimize")
input_dim = 529
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-04-24 00:14:13,871] A new study created in memory with name: no-name-db5d61e9-6c01-4996-9377-bb65e303e8a3



EPOCH 1 of 3

Training loss: 2.696290456421547
Validation loss: 0.5742513978973441

EPOCH 2 of 3

Training loss: 2.6369127591408916
Validation loss: 0.5742513978973441

EPOCH 3 of 3

Training loss: 2.680440048955507


[I 2023-04-24 00:15:05,736] Trial 0 finished with value: 0.5742513978973441 and parameters: {'lr': 0.0002056341946070166, 'weight_decay': 1.3308468823158127e-06, 'hidden_size': 76, 'dropout_rate': 0.8858624577083478}. Best is trial 0 with value: 0.5742513978973441.


Validation loss: 0.5742513978973441
{'lr': 0.0002056341946070166, 'weight_decay': 1.3308468823158127e-06, 'hidden_size': 76, 'dropout_rate': 0.8858624577083478}


In [198]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"])
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params.pt")
# save the trained model
torch.save(model, "faceoffs_best_model.pt")

{'lr': 0.0002056341946070166, 'weight_decay': 1.3308468823158127e-06, 'hidden_size': 76, 'dropout_rate': 0.8858624577083478}
Epoch 1 Training loss: 0.9235151579003718
Epoch 2 Training loss: 0.6508098351690224
Epoch 3 Training loss: 0.6168516320728902


In [199]:
# Evaluate on test set
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y.unsqueeze(1))
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")



Test Loss: 0.5807
True labels: [0, 0, 0, -1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -2, 0, 0, 0, 0, 0, 0, 0, -1, -2, 2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 2, 0, 0, 0, 0, -1, -2, 0, 0, -1, -1, -1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, -2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [200]:
for batch, (X, y) in enumerate(train_loader):
    print(y)

tensor([-2,  0,  0,  0,  4,  0,  1,  0,  0,  0,  0, -1,  0,  0, -1,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  3, -1, -2,
         1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  3, -1,  0,  0,
        -1, -2,  0, -1, -2,  0,  0,  0,  0,  2])
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0, -1,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  2, -1,  0,  0,  0, -1,  0,  0,  0, -1,
         0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,
         0,  0,  0, -2,  0,  0,  0,  0, -1,  0])
tensor([ 0,  3,  1,  0, -1,  0,  0, -2,  0, -1, -2,  0,  0,  0,  0,  0,  0, -2,
         0,  0,  0,  2,  0,  1,  1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  0,  0,  0, -2,  0, -1,  0,  0,  0,  0,  0,  0,  0,
         0,  0, -2,  0,  0,  0,  0,  0,  0,  0])
tensor([-2,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -2,
         0,  3,  0, -2,  0,  0,  0,  0,  0,  0,  0, -

In [118]:
print("Start Training...")
num_epochs = 200
training_losses = np.array([])
validation_losses = np.array([])
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
      X = X.float().to(device)
      y = y.float().to(device)
      prediction = model(X).to(device)
      optimizer.zero_grad()
      loss = loss_func(prediction, y.unsqueeze(1))
      loss.backward()
      optimizer.step()
      train_loss += loss.float().item()
      count += 1
    scheduler.step()
    training_losses = np.append(training_losses, train_loss / count)
    print("Training loss:", train_loss / count)
    model.train(False)

    ########################### Validation #####################################
    # TODO: Design your own validation section

    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
      for X, y in valid_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y)
        val_loss += loss.float().item()
        count += 1
    print("Validation loss:", val_loss / count)
    validation_losses = np.append(validation_losses, val_loss / count)

torch.save(model, "faceoffs.pt")


Start Training...

EPOCH 1 of 200

Training loss: 9.433583816783952e-11
Validation loss: 9.993852743323136e-10

EPOCH 2 of 200

Training loss: 1.6394327590896907e-08
Validation loss: 6.276212914568507e-11

EPOCH 3 of 200

Training loss: 1.956667736174093e-08
Validation loss: 5.3154151204911186e-08

EPOCH 4 of 200

Training loss: 7.113325624878379e-08
Validation loss: 5.65364305829923e-10

EPOCH 5 of 200

Training loss: 1.8580883692973614e-07
Validation loss: 1.164440860885496e-10

EPOCH 6 of 200

Training loss: 2.5086011643174e-07
Validation loss: 9.016696314157015e-10

EPOCH 7 of 200

Training loss: 1.9878597500110104e-07
Validation loss: 6.091113381670632e-08

EPOCH 8 of 200

Training loss: 9.346650389684619e-07
Validation loss: 2.013606519196483e-11

EPOCH 9 of 200

Training loss: 6.643579821118308e-08
Validation loss: 1.8888908709329277e-10

EPOCH 10 of 200

Training loss: 2.4145597567285983e-11
Validation loss: 1.233011979899692e-11

EPOCH 11 of 200

Training loss: 8.3005211719456

KeyboardInterrupt: 

In [126]:
class VanillaNeuralNet(nn.Module):

    def __init__(self):
        super(VanillaNeuralNet, self).__init__()
        self.fc1 = nn.Linear(529, 750) # 529 comes from printing and looking at input shape
        self.fc2 = nn.Linear(750, 750)
        self.fc3 = nn.Linear(750, 500)
        self.fc4 = nn.Linear(500, 500)
        self.fc5 = nn.Linear(500, 500)
        self.fc6 = nn.Linear(500, 500)
        self.fc7 = nn.Linear(500, 250)
        self.fc8 = nn.Linear(250, 100)
        self.fc9 = nn.Linear(100, 50)
        self.fc10 = nn.Linear(50, 1) # output singular prediction
    
    def forward(self, x):
        x_1 = self.fc1(torch.nn.functional.relu(x))
        x_2 = self.fc2(torch.nn.functional.relu(x_1))
        x_3 = self.fc3(torch.nn.functional.relu(x_2))
        x_4 = self.fc4(torch.nn.functional.relu(x_3))
        x_5 = self.fc5(torch.nn.functional.relu(x_4))
        x_6 = self.fc6(torch.nn.functional.relu(x_5))
        x_7 = self.fc7(torch.nn.functional.relu(x_6))
        x_8 = self.fc8(torch.nn.functional.relu(x_7))
        x_9 = self.fc9(torch.nn.functional.relu(x_8))
        x_10 = self.fc10(torch.nn.functional.relu(x_9))
        return x_10

In [75]:
class FaceoffsDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.data.iloc[idx, 1:]
        data = np.array([data])
        data = data.reshape(-1, 2)

        if self.transform:
            data = self.transform(data)

        return data

In [79]:
class CreateDataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    
    def __getitem__(self, index):
        sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32), 
                    'label': torch.tensor(self.y[index], dtype=torch.float32)}

        return sample


    def __len__(self):
        return len(self.x)

In [80]:
training_dataset = CreateDataset(X_train, y_train)
validation_dataset = CreateDataset(X_val, y_val)
print(training_dataset.x.shape)
print(training_dataset.y.shape)
print(validation_dataset)

torch.Size([60387, 529])
torch.Size([60387])


In [83]:
import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
        torch.cuda.empty_cache()

model = VanillaNeuralNet().to(device)
learning_rate = 0.01
train_batch_size = 300
validation_batch_size = 300
train_dataloader = DataLoader(training_dataset, batch_size=train_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=validation_batch_size, shuffle=False)
## Initialize Optimizer and Learning Rate Scheduler
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()
print("Start Training...")
num_epochs = 200
training_losses = np.array([])
validation_losses = np.array([])
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    train_loss = 0
    count = 0
    for entity in enumerate(train_dataloader):
      print(entity)
      batch, (X, y) = entity
      X = X.to(device)
      y = y.to(device)
      prediction = model(X).to(device)
      optimizer.zero_grad()
      loss = loss_func(prediction, y)
      loss.backward()
      optimizer.step()
      train_loss += loss.float().item()
      count += 1
    scheduler.step()
    training_losses = np.append(training_losses, train_loss / count)
    print("Training loss:", train_loss / count)
    model.train(False)

    ########################### Validation #####################################
    # TODO: Design your own validation section

    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
      for X, y in validation_dataloader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y)
        val_loss += loss.float().item()
        count += 1
    print("Validation loss:", val_loss / count)
    validation_losses = np.append(validation_losses, val_loss / count)

torch.save(model, "colorization_model_1.pt")


Start Training...

EPOCH 1 of 200

(0, {'feature': tensor([[ 1.0080,  1.0077,  0.2188,  ..., -0.7299, -1.4459,  0.5050],
        [ 0.3061,  0.3063, -0.7021,  ...,  0.0053, -0.0244,  0.0418],
        [ 0.3061,  0.3063, -0.7021,  ...,  0.0053, -0.0244,  0.0418],
        ...,
        [ 1.0077,  1.0077,  0.1131,  ...,  0.9856,  0.6864,  0.9682],
        [ 0.3061,  0.3063, -0.7021,  ...,  0.0053, -0.0244,  0.0418],
        [-1.7980, -1.7979,  1.2013,  ..., -1.3425, -0.9128, -1.3477]]), 'label': tensor([0.0000, 0.0092, 0.0092, 0.0409, 0.0099, 0.0513, 0.0092, 0.0092, 0.0092,
        0.0092, 0.0000, 0.2012, 0.0092, 0.0195, 0.0092, 0.0000, 0.0092, 0.2178,
        0.1906, 0.0092, 0.0092, 0.0000, 0.0814, 0.0092, 0.0245, 0.0000, 0.0092,
        0.0092, 0.0000, 0.1732, 0.0092, 0.0092, 0.0092, 0.0080, 0.0000, 0.2021,
        0.0217, 0.0092, 0.0092, 0.0000, 0.0092, 0.0000, 0.0092, 0.0165, 0.0299,
        0.0092, 0.0262, 0.0092, 0.0319, 0.0092, 0.0028, 0.0092, 0.0092, 0.0092,
        0.0092, 0.0195, 0

C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32),
C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(self.y[index], dtype=torch.float32)}


AttributeError: 'str' object has no attribute 'to'

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
        torch.cuda.empty_cache()

model = VanillaNeuralNet().to(device)
learning_rate = 0.01
train_batch_size = 300
validation_batch_size = 300
train_dataloader = DataLoader(training_dataset, batch_size=train_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=validation_batch_size, shuffle=False)
## Initialize Optimizer and Learning Rate Scheduler
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

In [64]:
print("Start Training...")
num_epochs = 200
training_losses = np.array([])
validation_losses = np.array([])
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_dataloader):
      X = X.to(device)
      y = y.to(device)
      prediction = model(X).to(device)
      optimizer.zero_grad()
      loss = loss_func(prediction, y)
      loss.backward()
      optimizer.step()
      train_loss += loss.float().item()
      count += 1
    scheduler.step()
    training_losses = np.append(training_losses, train_loss / count)
    print("Training loss:", train_loss / count)
    model.train(False)

    ########################### Validation #####################################
    # TODO: Design your own validation section

    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
      for X, y in validation_dataloader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y)
        val_loss += loss.float().item()
        count += 1
    print("Validation loss:", val_loss / count)
    validation_losses = np.append(validation_losses, val_loss / count)

torch.save(model, "colorization_model_1.pt")

Start Training...

EPOCH 1 of 200



C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32),
C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(self.y[index], dtype=torch.float32)}


AttributeError: 'str' object has no attribute 'to'